In [ ]:
# CNN on MNIST

import torch 
import torch.nn as nn
import torch.optim as optimizer
import torchvision
import numpy as np

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_dataset = torchvision.datasets.MNIST(root='../datasets/', train=True,
                                          transform = torchvision.transforms.ToTensor(),
                                          download = False)
test_dataset = torchvision.datasets.MNIST(root='../datasets/', train=False,
                                          transform = torchvision.transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=64,
                                          shuffle =True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=64,
                                          shuffle =False)


In [ ]:
num_classes = 10
l_rate = 0.01
class Convolution(nn.Module):
    def __init__(self, num_classes=10):
        super(Convolution, self).__init__()
        
        self.layer1 = nn.Sequential(
        nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2) )
        
        self.layer2 = nn.Sequential(
        nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.Dropout(),
        nn.MaxPool2d(kernel_size=2, stride=2)    )
        
        self.fc = nn.Linear(7*7*32, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out    

In [ ]:
model = Convolution(num_classes).to(device)

In [ ]:
loss = nn.CrossEntropyLoss()
optimiz = optimizer.SGD(model.parameters(), lr=l_rate)

In [ ]:
steps = len(train_loader)
n_epochs = 2
for e in range(n_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        loss_val = loss(outputs, labels)
        
        optimiz.zero_grad()
        loss_val.backward()
        optimiz.step()
        if (i+1)%5==0:
            print('Epoch [{}/{}], step [{}/{}], loss ------> {:.3f}'
                  .format(e+1, n_epochs, i, steps, loss_val.item()))


In [ ]:
model.eval()     ##########      To let batchnorm know, it is test mode
with torch.no_grad():
    tot = 0
    corr = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        output = model(images)
        _, pred = torch.max(output.data, 1)
        tot+=labels.size(0)
        corr+=(pred==labels).sum().item()
    print('Test Accuracy of the model on test images: {} %'.
           format(100 * corr / tot))

torch.save(model.state_dict(), '../datasets/models/torch104.ckpt')